In [ ]:
#This code is used to evolve and train networks to give high differentiation
#Also looks at family history, PCA orbits and robustness (both need a fix) 

In [ ]:
using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using DelimitedFiles


Data_Name = "High_ODE_No_Split"
mkpath("Data_All/" * Data_Name)
#Main variables to change:
SplitTime::Float32 = 25 #Time before a splitting event happens
N::Int32 = 4 #How many copies of same network we have
K::Int32 = N #Depricated parameter, do not change
P = 32 #How many different networks will we use in the evolution run
Generations = 750 #How many generations in the evolution run
Splits::Int32 = 4 #How often will our cell split
Splitting_Strength = 0.0f0
M::Int32 = 100 #This is the amount of genes we want in our network
#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!
Perturb_Radius = 0 # Radius of initial condition perturbations
Stochastic_Noise = true
Noise = 0.001

#Constant and plotting tools
a::Int32 = 1.0   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4  #Speed at which epigenetic factor changes
InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Perturb_Counter = 0 #For the initial condition perturbations, leave to keep same perturbations
#Saved_Data = [1,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150] #These are the generations where we want to save some orbits for

Saved_Data = [1,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300] #These are the generations where we want to save some orbits for
Saved_Data = [10*i for i=1:75]
Saved_Data = vcat(1,Saved_Data)

Saved_Generations = [2000,5000,8000,12000] #Something is broken about this, try to fix later
#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
KArray[1:10] .= 0 #Here we can manually adjust which genes will have diffusion

function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,Splitting_Strength)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end
    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end


function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du .= Noise
    
end



function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end

function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)

    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])                
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        sol = solve(prob,LambaEulerHeun())
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])                
    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:K]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end



function Earth_Movers_Distance_1D(Vector1,Vector2)

    #Takes 2 distribution vectors containing bins with a position and their %
    Length_V = length(Vector1[1,:]) #Both should be same length anyways
    EMD = zeros(Length_V+1) #We add an extra one because of the following loop
    
    for i =1:Length_V
        EMD[i+1] = Vector1[2,i] - Vector2[2,i] + EMD[i]
    end
    EMD_Distance = 0
    
    for i =1:Length_V-1
        EMD_Distance += abs(EMD[i+1])*(Vector1[1,i+1] - Vector1[1,i]) #We take moved distribution times moved distance

    end

    
    return EMD_Distance
end


function Robustness_Distribution_Scoring()
    
    
    #Here we create a very general robustness measure
    #This is done by taking the PCA components over all the different generations
    #Then we categorise all cell types and measure how similar their distribution
    #is in a given network

    #!!!!!!! This robustness does not consider cell type distances
    
    Robustness_Score = zeros(Generations,P)
    for Genere=1:Generations  #We want to calculate this for all generations
        for i = 1:P #We have to repeat this for all different networks in a generation
            #This is used to find the maximum of different cell types along every dimension in this network
            Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
            for j=1:N
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                for k=1:PCA_Dimensions
                    if Int.(length(countmap(Transformed[k,:]))) >  Dimensions_Distribution[k]
                        Dimensions_Distribution[k] = Int.(length(countmap(Transformed[k,:])))
                    end
                end   
            end

            #Now we have the max dimensions, now we want to make all distributions
            #Of this specific network

            Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
            #print(Dimensions_Distribution) #if we wanna visualise

            Cell_Distribution_Collection = Array{Array{Float64, PCA_Dimensions}}(undef, N)
            Vectorised_Cell_Distribution_Collection = Array{Vector{Float64}}(undef, N)

            for j=1:N #Now we collect all the distributions
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!
                Cell_Distribution_Collection[j] = PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)

                #Now we want to convert all of these into vectors to apply Jensen-Shannon
                Vectorised_Cell_Distribution_Collection[j] = sort(vec(Cell_Distribution_Collection[j]),rev=true)
            end

            #JS-divergence
            for tau = 1:N-1
                for pau = tau:N
                    Robustness_Score[Genere,i] += js_divergence(Vectorised_Cell_Distribution_Collection[tau],Vectorised_Cell_Distribution_Collection[pau])
                end
            end


        end
    end
    return Robustness_Score
end



function EMD_Generational_Calculator()
    
    #Here we will create a Robustness measure that takes into account how far
    #away different cell types are for the same network.
    #we will do this by creating a PCA analysis for EVERY single network
    #for all the different generations
    #(Alternatively we see if we can use the same PCA analysis within one generation)
    #We will create a one dimensional distribution and use our simplified
    #EMD algorithm
    
    Vectorised_Cell_Network_Collection = Array{Matrix{Float64}}(undef, N) #Here we save these distributions
    #This loops over copies of same network and makes all our distributions
    EMD_Distances = zeros(Generations)
    for generation_p = 1:Generations
        EMD_Dist = 0
        for P_Valu = 1:P
            FullNetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,:,:,:]),2^Splits*N,M)'
            Networkmodel= fit(PCA,FullNetworkBits;maxoutdim=1)
            Networkmodel_EigenV = eigvals(Networkmodel::PCA)
            for i=1:N
                #This creates the Distribution of a single individual
                #We start to apply this PCA to one of these individuals
                NetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,i,:,:]),2^Splits,M)'
                Transformed_Network = transform(Networkmodel,NetworkBits)
                Distribution_Netw = zeros(0)
                X_Values = zeros(0)
                Transformed_Network = round.(Transformed_Network,digits=4) #Otherwise ugly floating point numbers
                Length_Netw_Normal = 1/length(Transformed_Network)
                for i in Transformed_Network

                    if i in X_Values
                        Distribution_Netw[findfirst(==(i), X_Values)] += Length_Netw_Normal
                    else
                        X_Values = vcat(X_Values,i)
                        Distribution_Netw = vcat(Distribution_Netw,Length_Netw_Normal)
                    end
                end



                Combined_Distribution = zeros(2,length(X_Values))
                Combined_Distribution[1,:] = round.(X_Values/Networkmodel_EigenV,digits=4)
                Combined_Distribution[2,:] = Distribution_Netw
                Combined_Distribution = sortslices(Combined_Distribution,dims=2)

                Vectorised_Cell_Network_Collection[i] = Combined_Distribution
            end


            #Now we need to apply EMD to these distributions and save the result

            for tau = 1:N-1   #Preparing the vectors to have the same X values and length
                for pau = tau+1:N
                    Vector1 = Vectorised_Cell_Network_Collection[tau]
                    Vector2 = Vectorised_Cell_Network_Collection[pau]
                    for i in Vector1[1,:]
                        if !(i in Vector2[1,:])
                            Vector2 = hcat(Vector2, [i,0]) #Adding empty position
                        end
                    end
                    for i in Vector2[1,:]
                        if !(i in Vector1[1,:])
                            Vector1 = hcat(Vector1, [i,0]) #Adding empty position
                        end
                    end

                    Vector1 = (sortslices(Vector1,dims=2)) #Sorting them with the added X values
                    Vector2 = (sortslices(Vector2,dims=2))
                    EMD_Dist += Earth_Movers_Distance_1D(Vector1,Vector2) #Calculating EDM for 1D

                end
            end
        end
        EMD_Dist = 2*EMD_Dist/(P*N*(N-1))  #Normalisation
        EMD_Distances[generation_p] = EMD_Dist
    end
    return EMD_Distances
end


function Two_Generational_Plotter(Generation,Cut_Off,Duration,Length_Orbits,Opacity)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Shortest>size(Orbit_Array[i,j])[2]
                Shortest = size(Orbit_Array[i,j])[2] #Used later to check which data set is the shortest when plotting
            end

            Total_Length += size(Orbit_Array[i,j])[2]

        end
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2*8*2^Splits)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Cut_Off
                
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,1:Length_Orbits]
                end
            else
                Length_Orbits =  size(Orbit_Array[i,j])[2]
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,:]
                end
            end

            Collection_Index += Length_Orbits*2^Splits
        end
    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,2,Amount_Copies,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:2
        for j=1:Amount_Copies

            for tau=1:2^Splits
                tester = Orbit_Array[i+2*(Generation-1),j][2+(tau-1)*M:1+tau*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i,j,tau,:,:] = Transformed
            end
        end
    end
    
    
    Colours = [:red,:orange,:maroon,:chocolate4]
    @gif for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1,1:k],PCA_Orbits_2Gen[1,1,1,2,1:k],PCA_Orbits_2Gen[1,1,1,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for j=1:8
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,j,tau,1,1:k],PCA_Orbits_2Gen[i,j,tau,2,1:k],PCA_Orbits_2Gen[i,j,tau,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        i=2
        for j=1:4
            colour = Colours[j]
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j-1,tau,1,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,2,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j,tau,1,1:k],PCA_Orbits_2Gen[i,2*j,tau,2,1:k],PCA_Orbits_2Gen[i,2*j,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)

            end
        end


    end
end




function Generational_Tracker(Family_Distance,Original_Tracking,Types)
    Cell_Generational_Tracking = deepcopy(Original_Tracking)
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    size(Types[1,:])


    TypeFam = [Types[1,:],1]
    TypeFam = hcat(TypeFam,[Types[1,:],2])
    TypeFam = hcat(TypeFam,[Types[1,:],3])
    TypeFam = hcat(TypeFam,[Types[1,:],4])


    TypeFam = [Types[1,:],1]
    Distances_Gen = zeros(length(Cell_Generational_Tracking[1,:]))
    Cell_Generational_Tracking[1,1] = 1
    for i=2:length(Cell_Generational_Tracking[1,:])
        NewFam = true
        for j=1:length(TypeFam[1,:]) 
            if sum(abs.(Types[i,:]-TypeFam[1,j])) < Family_Distance + 0.5
                if NewFam
                    NewFam = false
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                elseif Distances_Gen[j] > sum(abs.(Types[i,:]-TypeFam[1,j]))
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                end
            end
        end
        if NewFam
            Distances_Gen[i] = sum(abs.(Types[i,:]-TypeFam[1,1]))
            Cell_Generational_Tracking[1,i] = Int(length(TypeFam[1,:])+1)
            TypeFam = hcat(TypeFam,[Types[i,:],Int(length(TypeFam[1,:])+1)])

        end
    end

    Cell_Generational_Tracking = Cell_Generational_Tracking[:,sortperm(Cell_Generational_Tracking[1,:])]



    Amount_Types = Cell_Generational_Tracking[1,end]
    Top_Bound_Index = zeros(Amount_Types)
    for j=1:Amount_Types
        Top_Bound_Index[j] = findfirst(isequal(j),Cell_Generational_Tracking[1,:])
    end


    #print(Cell_Generational_Tracking)
    Copied_Tracker = deepcopy(Cell_Generational_Tracking[2:end,:])
    Copied_Tracker = Copied_Tracker/sum(Copied_Tracker[1,:])

    Boundaries = zeros(Generations,length(Copied_Tracker[1,:]))
    for beta in 2:length(Copied_Tracker[1,:])
        Copied_Tracker[:,beta] += Copied_Tracker[:,beta-1]
        Boundaries[:,beta] = Copied_Tracker[:,beta] - Copied_Tracker[:,beta-1]
    end

    Cell_Type_Plotter = plot(Copied_Tracker[:,1]; ribbon = (Copied_Tracker[:,1],LinRange(0, 0, Generations)))
    for beta in 2:length(Copied_Tracker[1,:])
        Cell_Type_Plotter = plot!(Copied_Tracker[:,beta]; ribbon = (Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    
    #If we want to see the individual cells
    #display(Cell_Type_Plotter)






    #Now we do the same but we group them in families


    Family_Boundaries = ones(Generations,Amount_Types)
    Family_Boundaries[:,1] = Copied_Tracker[:,Int(Top_Bound_Index[2]-1)].-0
    for beta in 2:Amount_Types-1

        Family_Boundaries[:,beta] = Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)] .- Copied_Tracker[:,Int(Top_Bound_Index[beta]-1)]
    end
    Family_Boundaries[:,end] = 1 .- Copied_Tracker[:,Int(Top_Bound_Index[Amount_Types]-1)]


    Cell_Type_Family_Plotter = plot(Copied_Tracker[:,Int(Top_Bound_Index[2]-1)]; ribbon = (Family_Boundaries[:,1],LinRange(0, 0, Generations)))
    for beta in 2:Amount_Types-1
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)]; ribbon = (Family_Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    Cell_Type_Family_Plotter = plot!(ones(Generations); ribbon = (Family_Boundaries[:,Amount_Types],LinRange(0, 0, Generations)),legend=false)

    if false
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,1],linestyle=:dash)
        for beta in 2:length(Copied_Tracker[1,:])
            Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,beta],linestyle=:dash)
        end
    end

    display(Cell_Type_Family_Plotter)
    savefig(Cell_Type_Family_Plotter,"Data_All/" * Data_Name * "/FamilyTracker_Dist="* string(Family_Distance) *  ".png")
    save("Data_All/" * Data_Name * "/FamilyTracker_Dist="* string(Family_Distance) *  ".jld", "Boundaries", Family_Boundaries, "Top_Bound_Index",Top_Bound_Index  )
end

function Generational_Tracker_Initialisation()
    TypesString = deepcopy(Cell_Generational_Tracking[1,:])
    TypesString = string.(TypesString,base=2,pad=100)
    Types = zeros(Int16,length(Cell_Generational_Tracking[1,:]),M)
    for i in 1:length(Cell_Generational_Tracking[1,:])
        for j=1:M
            Types[i,j] = (parse(Int16,TypesString[i][j]))
        end
    end
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    return Original_Tracking, Types
end

function Interpolator(Times,Orbits)
    Vectors = zeros(length(Times),801)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end

function Perturbed_Position(Initial,Radius,Perturb_Counter)
    rng = MersenneTwister(Perturb_Counter);

    Radius = Radius/sqrt(M)
    for i=1:N
        Initial[i,1,1,1:M] += 2*Radius*(rand(rng,Float32,M).-0.5); 
    end

    return Initial,Perturb_Counter+N
end

# # Initialisation 

In [ ]:
Evolutionary_Complexity = []
CArray_Collection = zeros(Float32,N,M)
PositionArray_Collection = zeros(Float32,N,2,1,M)
ParameterMatrix_Collection = zeros(Int32,N,M,M)
Final_Epigenetic_Factors = zeros(Float32,N,2^(Splits),M)
Sparse = true
#[110, 108, 71, 16, 123, 119, 116, 103]
for i=1:N
    RandomInitial = 400 + 71 #Seed for Initial conditions

    CArray_Collection[i,:], PositionArray_Collection[i,:,:,:], ParameterMatrix_Collection[i,:,:] = Setting_Up_Randomly(M,Sparse,MersenneTwister(RandomInitial))
    #Sets the initial conditions that we will be reusing in the simulation (Change RandomInitial for different values)
    
end

Initial_Unperturbed = deepcopy(PositionArray_Collection)
Random_Noise = 574 #seed for splitting noise

#Need to define outside for loop, otherwise it will be destroyed afterwards
Times = 0
Genes = 0


In [ ]:
test = 3
"abc"*string(test)*"def"

In [ ]:
KLDistance_temp = 0
BitDistance_temp =  0
Evolutionary_Complexity = []
Indices = 0
Information = zeros(1)
Ratio = zeros(1)
Cell_Type_Tracker = []
Cell_Collection = []
Evolutionary_Complexity = zeros(K)
Temporary = []

@showprogress 1 for tau in 1:1
    for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
        print(size(Final_Epigenetic_Factors[x,:,:]))
    end
    Indices,AllIndices,Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Type_Tracker)
    Cell_Amounts = Cell_Collection[2]
    print(size(Evolutionary_Complexity))
    Cell_Amounts = sort(Cell_Amounts,rev=true)
    if length(Cell_Amounts) > 1
        Ratio = Cell_Amounts[2]/Cell_Amounts[1]
    else
        Ratio = 0
    end   
    print(Cell_Amounts)
    Cell_Amounts /= N*2
    Information[tau] -= Cell_Amounts[1]*log(Cell_Amounts[1])
    for kappa in 2:length(Cell_Amounts)
        Information[tau] -= Cell_Amounts[kappa]*log(Cell_Amounts[kappa])
    end
    

end

In [ ]:
Base.summarysize(Orbit_Array)/(10^9)
#typeof(Orbit_Array)

# # Here we are playing around with some stuff, PCA orbits, results etc.
## This isn't very important for the rest

In [ ]:
BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
Average_Bits = zeros(M)
for i=1:N
    for j=1:2
        Average_Bits += BitterMatrix[i,j,:]
    end
end
Average_Bits /= N*2
Deviation = 0

for i=1:N
    for j=1:2
        Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2*N-1)
    end
end
print(Deviation)
print(Average_Bits)

In [ ]:
#print(length(Cell_Type_Tracker[:,1]))
#print(Cell_Type_Tracker[1])
print(Cell_Collection)

Epigenetic_Plot = zeros(N,2^Splits,M)
Epigenetic_Plot[1,:,:] = Final_Epigenetic_Factors[1,:,:]  #Reshapes the factors to plot easily

for i=1:N
    Epigenetic_Plot[i,:,:] = Final_Epigenetic_Factors[i,:,:]  #Reshapes the factors to plot easily

end


#plot(Epigenetic_Plot[15,:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
#xlabel!("Cell Position")
#ylabel!("Epigenetic Factor")


In [ ]:
plot(Epi_Plotter(Indices,8,Final_Epigenetic_Factors))

In [ ]:
RelevantGenes = []
for kappa in 1:M
    bob = 0
    for theta in 1:length(Cell_Type_Tracker[:,1])
        bob += Cell_Type_Tracker[theta][kappa]
    end
    if (bob != 0) && (bob != length(Cell_Type_Tracker[:,1]))
        push!(RelevantGenes,kappa)
    end
end
print(RelevantGenes)
Index1 = RelevantGenes[1]
Index2 = RelevantGenes[2]
Index3 = RelevantGenes[3]

In [ ]:
Index1 = RelevantGenes[1]
Index2 = RelevantGenes[2]
Index3 = RelevantGenes[3]
x=1
Tau = 1
@gif for i=1:300
    plot(Orbit_Array[Tau,x][1][2][Index1,1:i],Orbit_Array[Tau,x][1][2][Index2,1:i],Orbit_Array[Tau,x][1][2][Index3,1:i],xlims=(-1,1),ylims=(-1,1),zlims=(-1,1),legend=false)
    for j=1:100
        plot!(Orbit_Array[Tau,x+j][1][2][Index1,1:i],Orbit_Array[Tau,x+j][1][2][Index2,1:i],Orbit_Array[Tau,x+j][1][2][Index3,1:i])
    end
end

In [ ]:
j=1
PCAComponents = [Orbit_Array[1,j][1][2][RelevantGenes[1],:] Orbit_Array[1,j][1][2][RelevantGenes[2],:]]
for tau in 3:length(RelevantGenes)
    PCAComponents = [PCAComponents Orbit_Array[1,j][1][2][RelevantGenes[tau],:]]
end
print(size(PCAComponents))
for j=2:N
    PCAComponents_Adder = [Orbit_Array[1,j][1][2][RelevantGenes[1],:] Orbit_Array[1,j][1][2][RelevantGenes[2],:]]
    for tau in 3:length(RelevantGenes)
        PCAComponents_Adder = [PCAComponents_Adder Orbit_Array[1,j][1][2][RelevantGenes[tau],:]]
    end
    PCAComponents = vcat(PCAComponents,PCAComponents_Adder)
end
print(size(PCAComponents))
PCAComponents = PCAComponents'
model= fit(PCA,PCAComponents;maxoutdim=3)

In [ ]:
Transformed = transform(model,PCAComponents)
ComponentLength = length(Orbit_Array[1,1][1][2][RelevantGenes[1],:])
print(ComponentLength)
#PCAPlots = plot(Transformed'[1:ComponentLength,1],Transformed'[1:ComponentLength,2],Transformed'[1:ComponentLength,3])
@gif for i=1:400
    PCAPlots = plot(Transformed'[1:1+i,1],Transformed'[1:1+i,2],Transformed'[1:1+i,3],legend=false)

    ComponentLength = length(Orbit_Array[1,1][1][2][RelevantGenes[1],:])
    for j=2:N
        NewCLength = length(Orbit_Array[1,j][1][2][RelevantGenes[1],:])
        PCAPlots = plot!(Transformed'[ComponentLength+1:ComponentLength+1+i,1],Transformed'[ComponentLength+1:ComponentLength+1+i,2],Transformed'[ComponentLength+1:ComponentLength+1+i,3],legend=false)
        ComponentLength += NewCLength
    end
end
#print(Transformed[2,:])




Length_Loop = length(SplitTime_Array)
Differentiations = zeros(Float32,Length_Loop)
for i=1:Length_Loop
    Differentiations[i] = mean(Evolutionary_Complexity[i])
end
Highest_Score = sum(Information)
print(Highest_Score)
abraca = plot(SplitTime_Array,Information)
display(abraca)
plot(SplitTime_Array,Differentiations.-1,label="Differentiations")
#plot!(SplitTime_Array,Cell_Types_Vector,label=L"1-I(Differntiated)")
#Removed this
plot!(SplitTime_Array,Ratio,label=L"Ratio = \frac{N_1}{N_2}")

ylims!(0,1.2)


# # This is the actual running of the simulation

In [ ]:
Saved_Generations = [90000,119000] #Something is broken about this, try to fix later

Original_Network = zeros(Int32,M,M)
Original_Network = ParameterMatrix_Collection[1,:,:]
MutateCounter = 100
Cells_To_Add = []

Final_Epigenetic_Factors = zeros(Float32,P,N,2^(Splits),M)
History_Final_Epigenetic_Factors = zeros(Float32,Generations,P,N,2^(Splits),M)


#ParameterTensor_Collection[N,32,M,M]?
ParameterTensor_Collection = zeros(Int32,P,N,M,M)
for p=1:P
    Mutated_Network = Mutate_Parameters(Original_Network,M,1,MutateCounter)
    MutateCounter +=1
    for x=1:N
        ParameterTensor_Collection[p,x,:,:] = Mutated_Network
    end
end
Pathway = "Plots/"*string(today())
mkpath(Pathway)
mkpath(Pathway*"/Updated")
Information = zeros(P)
Adjusted_Information = zeros(Generations)

Cell_Generational_Tracking = Array{Int128}(undef, Generations+1, 0)
Score_Graph = zeros(Generations)
KLDistances = zeros(Generations)
BitDistances = zeros(Generations)


Amount_Copies = 8
Orbit_Array = Array{Matrix{Float16}}(undef, 2*length(Saved_Generations),Amount_Copies) #We save 8 copies of original, then 2 of each
Temporary_Collection_Array = Array{Matrix{Float16}}(undef,Amount_Copies) 
#of the 4 daughters so we have (8+8)*length(Saved_Generations)



@showprogress 1 for gamma=1:Generations #This is the loop for multiple mutation runs
    print("This is generation: ")
    print(gamma)
    print("        ")
    Cells_To_Add = []
    #Evolutionary_Complexity = []
    #Cell_Types_Vector = zeros(P,length(SplitTime_Array))
    #Information = zeros(P,length(SplitTime_Array))
    #Adjusted_Information = zeros(P)
    #Ratio = zeros(P,length(SplitTime_Array))
    #Cell_Type_Tracker = [] this is broken
    #Orbit_Array = Array{Tuple{Vector{Array},Vector{Array},Vector{Array},Vector{Array}}}(undef, length(SplitTime_Array),N)
    Evolutionary_Complexity = zeros(P,K)
    Information = zeros(P)
    Highest_Scor_Tracker = 0
    p_Index = 1
    for p = 1:P #Goes over different networks
        PositionArray_Collection, Perturb_Counter = Perturbed_Position(Initial_Unperturbed,Perturb_Radius,Perturb_Counter)

        for x = 1:N #Goes over copies of the same network
            
            #Later we want to make Orbit_Array save all orbits and then animate the highest scoring one
            Final_Epigenetic_Factors[p,x,:,:],Random_Noise,Temporary = Running_Network(CArray_Collection[1,:],ParameterTensor_Collection[p,x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
            if gamma in Saved_Generations
                if x<Amount_Copies+1
                   Temporary_Collection_Array[x] = Temporary
                end
            elseif gamma-1 in Saved_Generations
                if p<5 #We only save the first 4, these are the daughters of the previous best
                     if x<3 #We only save 2 copies per daughter
                        Orbit_Array[2*findfirst(==(gamma-1), Saved_Generations),2*(p-1)+x] = Temporary

                    end
                end
            end
        end
        Indices,AllIndices,Evolutionary_Complexity[p,:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors[p,:,:,:],Cell_Type_Tracker)
        KLDistances[gamma] += 10*KLDistance_temp/P
        BitDistances[gamma] += BitDistance_temp/P
        push!(Cells_To_Add,Cell_Collection)
        Cell_Amounts = Cell_Collection[2]
        Cell_Amounts = sort(Cell_Amounts,rev=true)
        if p==1 #We plot it here just to see a bit
            display(plot(Epi_Plotter(Indices,4,Final_Epigenetic_Factors[p,:,:,:])))
        end
        Cell_Amounts /= (N*2^Splits)
        Information[p] -= Cell_Amounts[1]*log(Cell_Amounts[1])
        for kappa in 2:length(Cell_Amounts)
            Information[p] -= Cell_Amounts[kappa]*log(Cell_Amounts[kappa])
        end
        Adjusted_Information[gamma] += Information[p]/P
        #Information[p] = Information[p]*Gene_Variance(Final_Epigenetic_Factors[p,:,:,:],M,N) #We do regular information now
        
        if gamma in Saved_Generations #Use this to save specific orbits
            if Information[p] >= Highest_Scor_Tracker #Only saving highest score one
                p_Index = p
                Highest_Scor_Tracker = Information[p]
                for i=1:Amount_Copies
                    print("We are testing now")
                    print(i)
                    Orbit_Array[2*findfirst(==(gamma), Saved_Generations)-1,i] = deepcopy(Temporary_Collection_Array[i])
                    #96 breaks, on 20?
                    
                end
            end
        end
    end
    History_Final_Epigenetic_Factors[gamma,:,:,:,:] = Final_Epigenetic_Factors #This a big boy


    #plot(Epigenetic_Plot[15,:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    #xlabel!("Cell Position")
    #ylabel!("Epigenetic Factor")
    #print(Cells_To_Add[P][:])
    #print("yeet")
    #print(Cells_To_Add[P+1][1])
    for tau in 1:P
        Count = 1
        for iota in Cells_To_Add[tau][1]
            if Cell_Generational_Tracking==Array{Int128}(undef, Generations+1, 0)

                Adder = zeros(Int128,Generations+1)
                Adder[1] = iota
                Adder[gamma+1] = Cells_To_Add[tau][2][Count]

                Cell_Generational_Tracking = hcat(Cell_Generational_Tracking,Adder)

            elseif iota in Cell_Generational_Tracking[1,:]

                index = indexin(iota,Cell_Generational_Tracking[1,:])[1]
                Cell_Generational_Tracking[gamma+1,index] += Cells_To_Add[tau][2][Count]
            else
                Adder = zeros(Int128,Generations+1)
                Adder[1] = iota
                Adder[gamma+1] = Cells_To_Add[tau][2][Count]

                Cell_Generational_Tracking = hcat(Cell_Generational_Tracking,Adder) 

            end
            Count+=1
        end

    end
    
    #Use this print if we want to see cell types and amounts live
    #print(Cell_Generational_Tracking)
    
    
    #Length_Loop = length(SplitTime_Array)
    #Differentiations = zeros(Float32,P,Length_Loop)
    #for i=1:Length_Loop
    #    Differentiations[:,i] = mean(Evolutionary_Complexity[:,i,:],dims=2)
    #end
    
    #Change how we find highest score/selection of network, here also choose best P/4
    
    Sorted_Indices = sortperm(vec(sum(Information,dims=2)),rev=true)
    
    
    #Change these plots
    #abraca = plot(SplitTime_Array,Information[Int.(Sorted_Indices[1:Int(P/4)]),:]',colour="blue")
    #abraca = plot!(SplitTime_Array,Information[Int.(Sorted_Indices[Int(P/4)+1:P]),:]',colour="red",legend=false)

    #display(abraca)
    #Name = "/Information"*string(gamma)
    
    #savefig(Pathway*"/Updated"*Name)
    
    #We have to change these
    #plot(SplitTime_Array,Differentiations'.-1,label="Differentiations",legend=false)
    #Name = "/Parameters"*string(gamma)
    #savefig(Pathway*Name)
    #ylims!(0,1.2)

    if gamma in Saved_Data
        Location1 = "Data_All/" * Data_Name * "/Gen:"*string(gamma)*"_1.jld"
        Location2 = "Data_All/" * Data_Name * "/Gen:"*string(gamma)*"_2.jld"
        Location3 = "Data_All/" * Data_Name * "/Gen:"*string(gamma)*"_3.jld"
        Location4 = "Data_All/" * Data_Name * "/Gen:"*string(gamma)*"_4.jld"

        
        save(Location1, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterTensor_Collection[Sorted_Indices[1],1,:,:])
        save(Location2, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterTensor_Collection[Sorted_Indices[2],1,:,:])
        save(Location3, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterTensor_Collection[Sorted_Indices[3],1,:,:])
        save(Location4, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterTensor_Collection[Sorted_Indices[4],1,:,:])

    end

    if gamma in Saved_Generations #Here we check we got the right index
        print("Did we get the right index?")
        print(Sorted_Indices[1]==p_Index)
    end


    for p=1:Int(P/4)
        for pau=1:4
            Mutated_Network = Mutate_Parameters(ParameterTensor_Collection[Sorted_Indices[p],1,:,:],M,1,MutateCounter)
            MutateCounter +=1
            for x=1:N
                ParameterTensor_Collection[(p-1)*4+pau,x,:,:] = Mutated_Network
            end
        end
    end
    Score_Graph[gamma] = sum(Information)/P
    Generation_Plot_Update = plot(Score_Graph[1:gamma],label="Variational Score")
    Generation_Plot_Update = plot!(Adjusted_Information[1:gamma],label="Score")
    Generation_Plot_Update2 = plot(KLDistances[1:gamma],label="Distribution Robustness")
    Generation_Plot_Update2 = plot!(BitDistances[1:gamma],label="Cell Type Robustness")
    display(Generation_Plot_Update)
    display(Generation_Plot_Update2)



end    
plot(BitDistances[1:end],label="Cell Type Robustness")


In [ ]:
Generation_Plot_Update = plot(Score_Graph[1:Generations],label="Variational Score")
Generation_Plot_Update = plot!(Adjusted_Information[1:Generations],label="Score")
display(Generation_Plot_Update)
savefig(Generation_Plot_Update,"Data_All/" * Data_Name * "/FinalScore.png")


using DelimitedFiles

writedlm( "Data_All/" * Data_Name * "/FinalScore.csv",  Score_Graph, ',')

In [ ]:
#This is test data




save("Data_All/Data_Folder/myfile.jld", "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterMatrix_Collection[1,:,:])

# PCA orbits of 2 following generations

In [ ]:
Duration = 100
Length_Orbits = 1000
Opacity = 75
Two_Generational_Plotter(1,true,Duration,Length_Orbits,Opacity)

In [ ]:
print(Orbit_Array)

## Here we start the PCA analysis of the final cell states over the different generations to calculate robustness

In [ ]:
FinalPCAComponents = signbit.(-History_Final_Epigenetic_Factors)
FinalPCAComponents = reshape(FinalPCAComponents, Generations*P*N*2^Splits, M)
FinalPCAComponents = FinalPCAComponents'
print(size(FinalPCAComponents))
PCA_Dimensions = 2
model= fit(PCA,FinalPCAComponents;maxoutdim=PCA_Dimensions)

In [ ]:
#Just as a proof of concept we can show one with all (note it will look fucked up due to how it is transformed)
generation_p = 2
tester = reshape(signbit.(permutedims(-History_Final_Epigenetic_Factors[generation_p,:,:,:,:],[3,2,1,4])),2^Splits*P*N,M)'
Transformed = transform(model,tester)
Transform_Plot = plot(Transformed')
Transform_Plot = vline!(2^Splits*[i for i=1:N*P-1],colour=:"red",linealpha=:0.3)
display(Transform_Plot)



In [ ]:
#We create an array that collects all different networks and a set of same networks, used for the robustness stuff
Final_State_Collection_Robust = Array{Matrix{Float64}}(undef, P,N)
Cell_Distribution_Collection = 0 #Ignore this line, it's so I can debug


#Sum to average out over the different networks and plot in function of the generations
display(plot(sum(Robustness_Distribution_Scoring(),dims=2)))
#!!!!!!! This robustness does not consider cell type distances

    
    
#This one does!
EMD_Distances = EMD_Generational_Calculator()
plot(EMD_Distances)
    



# Variation in gene expression calculation (hourglass)

In [ ]:
Times = [i for i=1:1000]
Orbit_Collection = Orbit_Array[1,:]

#Collect all orbits in function of time,

#Maybe apply PCA to these orbits or keep them like this
Time_Orbital_Collection = zeros(Float16,length(Times),length(Orbit_Collection)*2^Splits,M)
Orbit_Counter=0
for Orbits in Orbit_Collection

    Interpolated_Vector = Interpolator(Times,Orbits) #We do linear interpolation to have all the same timesteps
    for i=1:2^Splits #We put the different splitted cells all together with the same networks.
        Time_Orbital_Collection[:,Orbit_Counter*2^Splits+i,:] = Interpolated_Vector[:,2+(i-1)*M:1+i*M]
    end
    Orbit_Counter += 1
end
#Calculate variance in gene expression in function of time

In [ ]:
Variance = zeros(200)
for i=1:200
    Variance[i] = sqrt(sum(std(Time_Orbital_Collection[i,:,:],dims=1).^2))
end
plot(Variance)
vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,legend=false)

## Generational cell type and family plotter

In [ ]:
Original_Tracking, Types = Generational_Tracker_Initialisation()

In [ ]:
Family_Distance = 18 #Families have to be more than this many genes apart
Generational_Tracker(Family_Distance,Original_Tracking,Types)

In [ ]:
Family_Distance = 10 #Families have to be more than this many genes apart
Generational_Tracker(Family_Distance,Original_Tracking,Types)

In [ ]:
Family_Distance = 12 #Families have to be more than this many genes apart
Generational_Tracker(Family_Distance,Original_Tracking,Types)

In [ ]:
Family_Distance = 14 #Families have to be more than this many genes apart
Generational_Tracker(Family_Distance,Original_Tracking,Types)

In [ ]:
Family_Distance = 20 #Families have to be more than this many genes apart
Generational_Tracker(Family_Distance,Original_Tracking,Types)

#This can probably be deleted, moved this to a function but just in caseI don't want to redo all of this

Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
for i=1:PCA_Dimensions
   Dimensions_Distribution[i] = Int.(length(countmap(Transformed[i,:])))
end
Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!


PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
#Here we will save all the values and use these for indexes later
B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
for j=1:PCA_Dimensions
    B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
    #different dimensions
    
    for k = 1:length(PCA_Dimensional_Values[1,:])
        PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
    end
end
#print(PCA_Dimensional_Values)   #If we wanna see them

Components_Length = length(Transformed[1,:]) 
for k in 1:Components_Length
    Index_PCA = zeros(PCA_Dimensions)
    
    for l=1:PCA_Dimensions
        Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
    end 
    Index_PCA = Int.(Index_PCA)
    
    if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
        Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
    elseif PCA_Dimensions==3
        Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
    elseif PCA_Dimensions==4
        Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
    elseif PCA_Dimensions==5
        Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
    else
    print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
    end
end

In [ ]:
test = "abra"
"al" * test * "ada"

In [ ]:
Gaussian = Normal{Float32}(0.0f0,0.25f0)
typeof(rand(Gaussian))

In [ ]:
Splitting_Strength = 0.25f0
Gaussian = Normal{Float32}(0.0f0,Radius)
typeof(rand(Gaussian))